<a href="https://colab.research.google.com/github/lupis30puc/bipm_text_analytics_exercises/blob/main/automated_similarity_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""import requests

url = "https://jamanetwork.com/journals/jamainternalmedicine/fullarticle/2766086"
page = requests.get(url, verify=False)
print(page.text)"""

'import requests\n\nurl = "https://jamanetwork.com/journals/jamainternalmedicine/fullarticle/2766086"\npage = requests.get(url, verify=False)\nprint(page.text)'

In [ ]:
"""import requests
try: 
  webpage = requests.get("https://jamanetwork.com/journals/jamainternalmedicine/fullarticle/2766086")
  print(webpage.text)
except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    """

'import requests\ntry: \n  webpage = requests.get("https://jamanetwork.com/journals/jamainternalmedicine/fullarticle/2766086")\n  print(webpage.text)\nexcept requests.exceptions.ConnectionError:\n    r.status_code = "Connection refused"\n    '

In [ ]:
import pandas as pd
import numpy as np
import re
import en_core_web_sm
import csv
import pickle
from google.colab import files

from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
from nltk.corpus import stopwords

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from scipy.spatial.distance import cosine, cdist

In [ ]:
meta = pickle.load(open('/content/meta_stemmed (1)', 'rb'))
meta.head()

,level_0,publish_time,title,abstract,date,year,language
0,4131,2020-01-09,Cocaine and amphetamine-regulated transcript p...,iolog cocain amphetamin gul transcript ppropep...,2020-01-09,2020,en
1,4195,2020-01-21,Pioneering Experience of Uniportal Video-Assis...,optim wai tat thorac scoliosi controvsi conven...,2020-01-21,2020,en
2,4310,2020-02-10,Epidemiological and clinical characteristics o...,sult surveil iolog charactist car hai eldli in...,2020-02-10,2020,en
3,4358,2020-02-07,Therapeutic strategies in an outbreak scenario...,origin psent spiratori curnt effort foc contai...,2020-02-07,2020,en
4,4485,2020-01-17,Single-Virus Tracking: From Imaging Methodolog...,im text un mechan assembl crucial pvent spad t...,2020-01-17,2020,en


In [ ]:
# resenting the index to fix an issue...
meta.reset_index(drop=True, inplace=True)
meta.head()

,level_0,publish_time,title,abstract,date,year,language
0,4131,2020-01-09,Cocaine and amphetamine-regulated transcript p...,iolog cocain amphetamin gul transcript ppropep...,2020-01-09,2020,en
1,4195,2020-01-21,Pioneering Experience of Uniportal Video-Assis...,optim wai tat thorac scoliosi controvsi conven...,2020-01-21,2020,en
2,4310,2020-02-10,Epidemiological and clinical characteristics o...,sult surveil iolog charactist car hai eldli in...,2020-02-10,2020,en
3,4358,2020-02-07,Therapeutic strategies in an outbreak scenario...,origin psent spiratori curnt effort foc contai...,2020-02-07,2020,en
4,4485,2020-01-17,Single-Virus Tracking: From Imaging Methodolog...,im text un mechan assembl crucial pvent spad t...,2020-01-17,2020,en


In [ ]:
target_df= pickle.load(open('smoking_stemmed', 'rb'))
target_df.head()

,Date,title,abstract
0,2020-05-22,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
1,2020-05-19,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
2,2020-05-19,Risk factors for adverse clinical outcomes in ...,hetogen vari prognosi charactist cours off gui...
3,2020-05-16,Severe obesity is associated with higher in-ho...,york curnt epicent undps minor pval obes appea...
4,2020-05-15,"Clinical Characteristics, Risk Factors and Out...",kuwait prior port ri mass scen incom travel ri...


In [ ]:
model = pickle.load(open('/content/d2v_model_saved.pkl', 'rb'))
d2v_target = pickle.load(open('/content/d2v_target.pkl', 'rb'))

In [ ]:
similars = model.docvecs.most_similar(positive=[d2v_target[1]], topn=3)
print(similars)

[(10970, 0.9892420768737793), (7054, 0.7739754319190979), (592, 0.7703381776809692)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
# Making a function for getting the top similar to add to the target table

def get_similar_docs(target_df, meta_df, d2v_model, d2v_target):
  """
  This function takes:
  [1] a target table dataframe
  [2] the metadata table dataframe
  [3] doc2vec model based on the metadata abstracts
  [4] doc2vec model of the target table obtained with the metadata doc2vec model

  Both the target and the metadata tables should contain columns for title and abstract.

  For this function to run successfully, 
  the following packages need to be installed:
   from gensim.models.doc2vec import Doc2Vec
   import pandas as pd

  At the end it prints the value count of the final dataframe that contains the following columns:
  ('original_index', 'original_db', 'similarity_percentage', 'title', 'abstract');
  and returns this mentioned dataframe.
  """

# Run the similarity test assuming all titles are in the filtered dataset:
  similar_to_target = []
  for i in range(len(target_df.title)):
    sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[i]], topn=3)
    #this way the list could be used to create a dataframe
    similar_to_target.append([i, 'target', 1, target_df.title[i], target_df.abstract[i]])
    similar_to_target.append([sim_test[0][0], 'most similar', sim_test[0][1], meta_df.title[sim_test[0][0]], meta_df.abstract[sim_test[0][0]]])
    similar_to_target.append([sim_test[1][0], 'second most similar', sim_test[1][1], meta_df.title[sim_test[1][0]], meta_df.abstract[sim_test[1][0]]])                        
    similar_to_target.append([sim_test[2][0], 'third most similar', sim_test[2][1], meta_df.title[sim_test[0][0]], meta_df.abstract[sim_test[0][0]]])

    # creating a dataframe with the top 3 most similar docs of the target ones!
    df_colum = ['original_index', 'original_db', 'similarity_percentage', 'title', 'abstract']
    similar_to_target_df = pd.DataFrame(similar_to_target, index=range(len(similar_to_target)), columns=df_colum)
    new_docs_target_df = similar_to_target_df.drop_duplicates(subset='title', keep="first", inplace=False)
  
  print(new_docs_target_df.original_db.value_counts())
  return similar_to_target_df, new_docs_target_df

In [ ]:
smoke_similar_docs = get_similar_docs(target_df, meta, model, d2v_target)

target                 30
second most similar    29
most similar            1
Name: original_db, dtype: int64


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
smoke_similar_docs.head()

,original_index,original_db,similarity_percentage,title,abstract
0,0,target,1.000000,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
2,6336,second most similar,0.874410,Characteristics and predictors of hospitalizat...,knowledg individu ord target stgi gen appropri...
4,1,target,1.000000,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
6,7054,second most similar,0.773975,"Delirium Incidence, Duration and Severity in C...",delirium incid durat intens ca known obsv larg...
8,2,target,1.000000,Risk factors for adverse clinical outcomes in ...,hetogen vari prognosi charactist cours off gui...


### Trying the whole process for diabetes

In [ ]:
diabetes_df= pickle.load(open('diabetes_stemmed', 'rb'))
diabetes_df.head()

,Date,title,abstract
0,2020-06-30,Prevalence and severity of corona virus diseas...,port decemb outbak known spad charactist publi...
1,2020-06-30,Clinical features and short-term outcomes of 2...,late decemb outbak spiratori emg iolog featur ...
2,2020-06-03,Plasma albumin levels predict risk for nonsurv...,explor biomark progss mati mod vsu vsu vsu sul...
3,2020-06-01,COVID-19 with Different Severities: A Multicen...,nal concn charactist comput tomographi im tatm...
4,2020-06-01,Characteristics and Outcomes of COVID-19 Patie...,york citi nyc born ga burden charactist racial...


In [ ]:
corpus_diabetes= [doc.split() for doc in list(diabetes_df.abstract)]
print(corpus_diabetes[0])

['port', 'decemb', 'outbak', 'known', 'spad', 'charactist', 'publish', 'cent', 'medlin', 'sinom', 'embas', 'cochran', 'librari', 'iolog', 'sult', 'incid', 'fev', 'cough', 'fatigu', 'dyspnea', 'pval', 'diabet', 'hyptens', 'cardiovascular', 'malign', 'complic', 'ard', 'rang', 'pool', 'estim', 'ard', 'aci', 'aki', 'shock', 'rang', 'pool', 'estim', 'spect', 'pcent', 'diabet', 'hyptens', 'spect', 'fev', 'pval', 'comorbid', 'hyptens', 'diabet', 'ard', 'aci', 'obstacl', 'tatment']


In [ ]:
# Applying the model to the target datset
d2v_diabetes = []
for i in range(len(corpus_diabetes)):
  model.random.seed(0)
  d2v_diabetes.append(model.infer_vector(corpus_diabetes[i], epochs=200))

d2v_diabetes[0]

array([-2.2191122 ,  4.9982414 , -3.5309076 , -0.8385328 , -0.793545  ,
       -3.8545969 , -6.0812864 ,  1.2535665 , -0.7832953 ,  0.91301864,
       -0.6431914 ,  3.103079  , -2.1967947 ,  0.65219283, -3.8912334 ,
       -2.4234843 , -3.1107209 ,  3.1117568 , -0.21674915,  3.9767525 ],
      dtype=float32)

In [ ]:
diabetes_similar_docs, diabetes_new_docs = get_similar_docs(diabetes_df, meta, model, d2v_diabetes)
diabetes_similar_docs.head(12)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


target                 61
second most similar    57
most similar            2
Name: original_db, dtype: int64


,original_index,original_db,similarity_percentage,title,abstract
0,0,target,1.000000,Prevalence and severity of corona virus diseas...,port decemb outbak known spad charactist publi...
1,10328,most similar,0.988107,Prevalence and severity of corona virus diseas...,port decemb outbak known spad charactist publi...
2,12221,second most similar,0.821373,Hypertension is associated with increased mort...,investig hyptens pneumonia pform systemat lita...
3,10864,third most similar,0.815593,Prevalence and severity of corona virus diseas...,port decemb outbak known spad charactist publi...
4,1,target,1.000000,Clinical features and short-term outcomes of 2...,late decemb outbak spiratori emg iolog featur ...
5,16220,most similar,0.991123,Clinical features and short-term outcomes of 2...,late decemb outbak spiratori emg iolog featur ...
6,6936,second most similar,0.946682,Clinical features and outcomes of 221 patients...,nal late decemb outbak spiratori offici name e...
7,12197,third most similar,0.825572,Clinical features and short-term outcomes of 2...,late decemb outbak spiratori emg iolog featur ...
8,2,target,1.000000,Plasma albumin levels predict risk for nonsurv...,explor biomark progss mati mod vsu vsu vsu sul...
9,11492,most similar,0.993745,Plasma albumin levels predict risk for nonsurv...,explor biomark progss mati mod vsu vsu vsu sul...
